<a href="https://colab.research.google.com/github/ttb-git/projects/blob/master/FrugalGPT_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎓 FrugalGPT: Better Quality and Lower Cost for LLM Apps

This notebook illustrates the FrugalGPT framework for _building LLM Applications with budget constraints._

Below, we will demonstrate two techniques offered by FrugalGPT to build cost-efficient LLM applications. The first techniques, LLMforAll, allows the users to query various LLM APIs via a unified inferface. The sceond one, LLMCascade, automates and optimizes the query process given a user-defined budget constraint.

NB: You are highly suggested to use accelerated hardware (GPU/TPU) to run this notebook.

## Installation
Let us start by installing FrugalGPT (if you haven't yet!).

In [ ]:
# set up the environment
%%capture
! git clone https://github.com/stanford-futuredata/FrugalGPT
%cd FrugalGPT
! pip install git+https://github.com/stanford-futuredata/FrugalGPT
! wget  https://github.com/lchen001/DataHolder/releases/download/v0.0.1/HEADLINES.zip
! unzip HEADLINES.zip -d strategy/
! rm HEADLINES.zip
! wget -P db/ https://github.com/lchen001/DataHolder/releases/download/v0.0.1/HEADLINES.sqlite
! wget -P db/ https://github.com/lchen001/DataHolder/releases/download/v0.0.1/qa_cache.sqlite

In [ ]:
%load_ext autoreload
%autoreload 2
import sys, json, copy
import logging
logging.disable(logging.CRITICAL)
sys.path.append("src/")

## Setup
Next, let us set up the environment and API keys. You do _not_ need API keys to run the notebook! They are only needed if you want to use FrugalGPT for your own queries.
#### NB: _For your own queries, not all API keys are needed, too. If you only want to leverage LLMs from, e.g., OpenAI and AI21, setting up API keys for them is sufficient._

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'
os.environ['AI21_STUDIO_API_KEY'] = 'AI21_STUDIO_API_KEY'
os.environ['COHERE_STUDIO_API_KEY'] = 'COHERE_STUDIO_API_KEY'
os.environ['TEXTSYNTH_API_SECRET_KEY'] = 'TEXTSYNTH_API_SECRET_KEY'
os.environ['ANTHROPIC_API_KEY'] = 'ANTHROPIC_API_KEY'
from IPython.display import display
import FrugalGPT
supported_LLM = FrugalGPT.getservicename()
print("supported LLMs:",supported_LLM)

supported LLMs: ['textsynth/gptneox_20B', 'textsynth/fairseq_gpt_13B', 'textsynth/gptj_6B', 'openai/text-davinci-002', 'openai/text-davinci-003', 'openai/text-curie-001', 'openai/text-babbage-001', 'openai/text-ada-001', 'openaichat/gpt-3.5-turbo', 'openaichat/gpt-4', 'ai21/j1-jumbo', 'ai21/j1-grande', 'ai21/j1-large', 'ai21/j2-ultra', 'ai21/j2-mid', 'ai21/j2-light', 'cohere/command', 'cohere/base', 'cohere/xlarge', 'cohere/medium', 'anthropic/claude-1', 'anthropic/claude-instant-1', 'anthropic/claude-1-100k']


## 1. LLMforAll: One interface for all LLM services
Let us first study an example for LLMforAll, an interface that unifies all existing services.  

In [ ]:
MyLLMforAll = FrugalGPT.LLMforAll()
query = "Question: Who is Matei Zaharia in 2023?\nAnswer:"
service_name = supported_LLM[-1]
genparams = FrugalGPT.GenerationParameter(max_tokens=50, temperature=0.1, stop=['\n\n\n\n'])
answer = MyLLMforAll.get_completion(query,service_name,genparams=genparams)
cost = MyLLMforAll.get_cost()
print("API:",service_name,"answer:",answer,"cost:",cost)

API: anthropic/claude-1-100k answer:  I apologize, but I do not actually know who Matei Zaharia will be in 2023 or specifics about his future career. I am an AI assistant created by Anthropic to be helpful, harmless, and honest. cost: 0.0016795999999999998


The above code snippet shows how to use LLMforAll. Its function get_completion gives a unified inferface for all LLMs: it takes the query, the generation parameters (such as temperature), and the service name as input, and then gives the corresponding generation. The cost can be obtained by calling get_cost().

In [ ]:
responses = MyLLMforAll.get_completion_allservice(query, supported_LLM, genparams=genparams)
print("full responses")
display(responses)

full responses


,service,answer,cost
0,textsynth/gptneox_20B,Matei Zaharia is a computer scientist who wor...,1.772400e-03
1,textsynth/fairseq_gpt_13B,Matei Zaharia is a Romanian politician and me...,7.590000e-04
2,textsynth/gptj_6B,The world’s first self-driving car.\n\nMatei ...,2.530000e-04
3,openai/text-davinci-002,Matei Zaharia is a Romanian-American computer...,1.080000e-03
4,openai/text-davinci-003,"In 2023, Matei Zaharia is a computer scientis...",1.300000e-03
5,openai/text-curie-001,Matei Zaharia will be 73 years old in 2023.,5.600000e-05
6,openai/text-babbage-001,Matei Zaharia is a Romanian-born American bus...,1.500000e-05
7,openai/text-ada-001,\n\nMatei Zaharia is a Russian-born American b...,2.000000e-05
8,openaichat/gpt-3.5-turbo,"As an AI language model, I cannot predict the ...",1.460000e-04
9,openaichat/gpt-4,"As an AI, I cannot predict the future or provi...",3.660000e-03


## 2. LLMCascade: Optimizing performance within budget constraints
Next let us use LLMCascade to automatically optimize the overall performance given a budget constraint.

### Example usage: predicting gold price trends from financial news
Let us first create a few NLP queries that asks LLM to predict gold price trends.

In [ ]:
dev = [['Q: april gold down 20 cents to settle at $1,116.10/oz\nA:', 'down','0'],
       ['Q: gold suffers third straight daily decline\nA:', 'down','1'],
       ['Q: Gold futures edge up after two-session decline\nA:', 'up','2'],
       ['Q: Dec. gold climbs $9.40, or 0.7%, to settle at $1,356.90/oz\nA:','up','3'],
       ['Q: Gold struggles; silver slides, base metals falter\nA:','up','4'],
       ['Q: feb. gold ends up $9.60, or 1.1%, at $901.60 an ounce\nA:','up','5'],
        ['Q: dent research : is gold\'s day in the sun coming soon?\nA:','none','6']
      ]
prefix = open('config/prompt/HEADLINES/prefix_e8.txt').read()
raw_data = copy.deepcopy(dev)
data = FrugalGPT.formatdata(dev,prefix)

Next let us load a LLMCascade instance.

In [ ]:
MyCascade = FrugalGPT.LLMCascade()
MyCascade.load(loadpath="strategy/HEADLINES/",budget=0.000665)

Let us take a look on LLMCascade's generation on one query.

In [ ]:
index = 1
query = data[index][0]
query_raw = raw_data[index][0]
genparams=FrugalGPT.GenerationParameter(max_tokens=50, temperature=0.1, stop=['\n'])
answer = MyCascade.get_completion(query=query,genparams=genparams)
cost = MyCascade.get_cost()
print("query:",query_raw)
print("FrugalGPT LLMCascade answer:",answer)

query: Q: gold suffers third straight daily decline
A:
FrugalGPT LLMCascade answer:  down


Now we can pass all the queries to both LLMCascade and vanilla GPT-4, and compare their performance.

In [ ]:
# batch generation
result = MyCascade.get_completion_batch(queries=data,genparams=genparams)
result_GPT4 = MyLLMforAll.get_completion_batch(queries=data,genparams=genparams,service_name='openaichat/gpt-4')
print("FrugalGPT LLMCascade generations")
display(result)
display(FrugalGPT.compute_score(result))
print("GPT-4 generations")
display(result_GPT4)
display(FrugalGPT.compute_score(result_GPT4))

FrugalGPT LLMCascade generations


,_id,answer,ref_answer,cost
0,0,down,down,0.000056
1,1,down,down,0.000054
2,2,up,up,0.000055
3,3,up,up,0.000058
4,4,down,up,0.000055
5,5,up,up,0.000057
6,6,none,none,0.000497


{'em': 0.8571428571428571, 'cost': 0.00011888571428571427}

GPT-4 generations


,_id,answer,ref_answer,cost
0,0,down,down,0.00678
1,1,down,down,0.00645
2,2,up,up,0.00651
3,3,up,up,0.00705
4,4,down,up,0.00657
5,5,up,up,0.00699
6,6,none,none,0.00663


{'em': 0.8571428571428571, 'cost': 0.006711428571428572}

Overall, FrugalGPT LLMCascade gives the same performance but incurs a much smaller cost. This data is of course quite small; Later we will see the evaluation on a larger dataset.

### Using FrugalGPT-LLMCascade for your own data
Interested in using FrugalGPT for your own data? No problem! The following code snippnet demonstrates how to do it.

The first thing is to load the training dataset.

In [ ]:
# load data
dev = FrugalGPT.loadcsvdata("data/HEADLINES/train.csv")
dev = dev[0:500]
prefix = open('config/prompt/HEADLINES/prefix_e8.txt').read()
data = FrugalGPT.formatdata(dev,prefix)

Second, specify the budget per query, and then train the model. In this example, only 3 APIs from OpenAI and AI21 are used, and thus setting up OpenAI and AI21 API keys is enough. Warning: The training process can take a while on large datasets!

In [ ]:
# train the model
MyCascade = FrugalGPT.LLMCascade()
service_names = ['openaichat/gpt-3.5-turbo','openaichat/gpt-4','ai21/j1-large']
user_budget = 0.002
result = MyCascade.train(data,budget=user_budget,service_names=service_names)

Epoch,Training Loss,Validation Loss,Accuracy
1,0.601300,0.605322,0.821549
2,0.592400,0.561019,0.821549
3,0.555900,0.492254,0.821549
4,0.382100,0.466782,0.821549
5,0.434900,0.471775,0.821549
6,0.375700,0.474937,0.821549
7,0.360800,0.457660,0.821549
8,0.302400,0.422048,0.821549


Epoch,Training Loss,Validation Loss,Accuracy
1,0.726400,0.707432,0.188552
2,0.707000,0.643969,0.895623
3,0.659100,0.545092,0.895623
4,0.500500,0.397062,0.895623
5,0.373900,0.334464,0.895623
6,0.393900,0.330068,0.895623
7,0.298800,0.317346,0.895623
8,0.268000,0.304869,0.895623


Epoch,Training Loss,Validation Loss,Accuracy
1,0.719700,0.707748,0.218855
2,0.707100,0.666228,0.791246
3,0.679000,0.608357,0.794613
4,0.625200,0.524273,0.794613
5,0.532600,0.484093,0.794613
6,0.526300,0.458705,0.794613
7,0.472700,0.458166,0.794613
8,0.426000,0.426861,0.794613


In [ ]:
# save to disk
MyCascade.save(savepath="strategy/TEST/")

Now the model has been saved to disk. You can load it as follows for future applications.

In [ ]:
MyCascade = FrugalGPT.LLMCascade()
MyCascade.load(loadpath="strategy/TEST/",budget=user_budget)

### Performance evaluation
Now let us evaluate the performance of FrugalGPT. We use LLMCascade on the HEADLINES dataset as an example.

In [ ]:
test = FrugalGPT.loadcsvdata("data/HEADLINES/test.csv")
prefix = open('config/prompt/HEADLINES/prefix_e8.txt').read()
data_eval = FrugalGPT.formatdata(test,prefix)
data_eval = data_eval[0:500]
genparams=FrugalGPT.GenerationParameter(max_tokens=50, temperature=0.1, stop=['\n'])
result_GPT4 = MyLLMforAll.get_completion_batch(queries=data_eval,genparams=genparams,service_name='openaichat/gpt-4')
print("GPT-4 generations")
display(result_GPT4)
display(FrugalGPT.compute_score(result_GPT4))
result = MyCascade.get_completion_batch(queries=data_eval,genparams=genparams)
print("FrugalGPT LLMCascade generations")
display(result)
display(FrugalGPT.compute_score(result))

GPT-4 generations


,_id,answer,ref_answer,cost
0,6556,up,up,0.00660
1,5832,none,none,0.00666
2,5618,none,none,0.00666
3,4205,down,down,0.00660
4,842,up,down,0.00654
...,...,...,...,...
495,4810,down,down,0.00660
496,6319,down,down,0.00693
497,7522,none,up,0.00654
498,4117,neutral,up,0.00669


{'em': 0.854, 'cost': 0.0066702}

FrugalGPT LLMCascade generations


,_id,answer,ref_answer,cost
0,6556,up,up,0.000360
1,5832,none,none,0.007464
2,5618,none,none,0.000804
3,4205,down,down,0.000360
4,842,up,down,0.000796
...,...,...,...,...
495,4810,down,down,0.000800
496,6319,down,down,0.000360
497,7522,up,up,0.000796
498,4117,up,up,0.000360


{'em': 0.868, 'cost': 0.0019392119999999996}

### Full performance evaluation
Now let us evaluate the performance of FrugalGPT. We use LLMCascade on the HEADLINES dataset as an example. Warning: This is also going to be slow.

First, we load the evaluation dataset and the LLMCascade.

In [ ]:
test = FrugalGPT.loadcsvdata("data/HEADLINES/test.csv")
prefix = open('config/prompt/HEADLINES/prefix_e8.txt').read()
data_eval = FrugalGPT.formatdata(test,prefix)
print("test data size:",len(data_eval))
MyCascade = FrugalGPT.LLMCascade()
MyCascade.load(loadpath="strategy/HEADLINES/",budget=0.000665)
#MyCascade.load(loadpath="strategy/HEADLINES/",budget=100)

test data size: 5000


And then let us evaluate it on the evaluation dataset.

In [ ]:
genparams=FrugalGPT.GenerationParameter(max_tokens=50, temperature=0.1, stop=['\n'])
result_GPT4 = MyLLMforAll.get_completion_batch(queries=data_eval,genparams=genparams,service_name='openaichat/gpt-4')
print("GPT-4 generations")
display(result_GPT4)
display(FrugalGPT.compute_score(result_GPT4))
result = MyCascade.get_completion_batch(queries=data_eval,genparams=genparams)
print("FrugalGPT LLMCascade generations")
display(result)
display(FrugalGPT.compute_score(result))

GPT-4 generations


,_id,answer,ref_answer,cost
0,6556,up,up,0.00660
1,5832,none,none,0.00666
2,5618,none,none,0.00666
3,4205,down,down,0.00660
4,842,up,down,0.00654
...,...,...,...,...
4995,8376,neutral,up,0.00651
4996,4242,down,down,0.00654
4997,663,down,up,0.00660
4998,6890,neutral,neutral,0.00648


{'em': 0.8556, 'cost': 0.006661218000000001}

FrugalGPT LLMCascade generations


,_id,answer,ref_answer,cost
0,6556,up,up,0.000055
1,5832,none,none,0.000499
2,5618,none,none,0.000499
3,4205,down,down,0.000055
4,842,up,down,0.000055
...,...,...,...,...
4995,8376,up,up,0.000055
4996,4242,down,down,0.000055
4997,663,down,up,0.000495
4998,6890,neutral,neutral,0.000054


{'em': 0.8594, 'cost': 0.00064688504}

Overall, LLMCascade achieves better performance than GPT-4 with a 10x smaller cost.